In [2]:
import numpy as np
%matplotlib notebook
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
import cv2
import sys
sys.path.append('../../')

# sort

In [3]:
from dl.data.utils.quads import sort_clockwise_topleft_numpy

quads=np.array([[1,0,0,1,1,0,0,1],[0.2,0.4,0.3,0.2,1,0.2,0.8,1],[0,0,1,1,0,0,1,1]])
sort_clockwise_topleft_numpy(quads)

array([[0. , 1. , 1. , 0. , 1. , 0. , 0. , 1. ],
       [0.3, 0.2, 1. , 0.2, 0.8, 1. , 0.2, 0.4],
       [0. , 0. , 1. , 1. , 1. , 1. , 0. , 0. ]])

In [45]:
# https://github.com/jiangxiluning/FOTS.PyTorch/blob/master/FOTS/utils/bbox.py
def sort_poly(p):
        min_axis = np.argmin(np.sum(p, axis = 1))
        p = p[[min_axis, (min_axis + 1) % 4, (min_axis + 2) % 4, (min_axis + 3) % 4]]
        if abs(p[0, 0] - p[1, 0]) > abs(p[0, 1] - p[1, 1]):
            return p
        else:
            return p[[0, 3, 2, 1]]

def sort_rectangle(poly):
    # sort the four coordinates of the polygon, points in poly should be sorted clockwise
    # First find the lowest point
    p_lowest = np.argmax(poly[:, 1])
    if np.count_nonzero(poly[:, 1] == poly[p_lowest, 1]) == 2:
        # 底边平行于X轴, 那么p0为左上角 - if the bottom line is parallel to x-axis, then p0 must be the upper-left corner
        p0_index = np.argmin(np.sum(poly, axis=1))
        p1_index = (p0_index + 1) % 4
        p2_index = (p0_index + 2) % 4
        p3_index = (p0_index + 3) % 4
        return poly[[p0_index, p1_index, p2_index, p3_index]], 0.
    else:
        # 找到最低点右边的点 - find the point that sits right to the lowest point
        p_lowest_right = (p_lowest - 1) % 4
        p_lowest_left = (p_lowest + 1) % 4
        angle = np.arctan(-(poly[p_lowest][1] - poly[p_lowest_right][1])/(poly[p_lowest][0] - poly[p_lowest_right][0]))
        # assert angle > 0
        if angle <= 0:
            print(angle, poly[p_lowest], poly[p_lowest_right])
        if angle/np.pi * 180 > 45:
            # 这个点为p2 - this point is p2
            p2_index = p_lowest
            p1_index = (p2_index - 1) % 4
            p0_index = (p2_index - 2) % 4
            p3_index = (p2_index + 1) % 4
            return poly[[p0_index, p1_index, p2_index, p3_index]], -(np.pi/2 - angle)
        else:
            # 这个点为p3 - this point is p3
            p3_index = p_lowest
            p0_index = (p3_index + 1) % 4
            p1_index = (p3_index + 2) % 4
            p2_index = (p3_index + 3) % 4

            return poly[[p0_index, p1_index, p2_index, p3_index]], angle

quad = np.array([0.2,0.4,0.3,0.2,1,0.2,0.8,1]).reshape(4,2)
print(sort_rectangle(np.array([1,0,0,1,1,0,0,1]).reshape(4,2)))
sort_rectangle(quad)

(array([[1, 0],
       [0, 1],
       [1, 0],
       [0, 1]]), 0.0)


(array([[0.3, 0.2],
        [1. , 0.2],
        [0.8, 1. ],
        [0.2, 0.4]]),
 -0.244978663126864)

In [46]:
def order_points_new(pts):
    # sort the points based on their x-coordinates
    xSorted = pts[np.argsort(pts[:, 0]), :]

    # grab the left-most and right-most points from the sorted
    # x-roodinate points
    leftMost = xSorted[:2, :]
    rightMost = xSorted[2:, :]

    # now, sort the left-most coordinates according to their
    # y-coordinates so we can grab the top-left and bottom-left
    # points, respectively
    leftMost = leftMost[np.argsort(leftMost[:, 1]), :]
    (tl, bl) = leftMost
    
    # if use Euclidean distance, it will run in error when the object
    # is trapezoid. So we should use the same simple y-coordinates order method.

    # now, sort the right-most coordinates according to their
    # y-coordinates so we can grab the top-right and bottom-right
    # points, respectively
    rightMost = rightMost[np.argsort(rightMost[:, 1]), :]
    (tr, br) = rightMost

    # return the coordinates in top-left, top-right,
    # bottom-right, and bottom-left order
    return np.array([tl, tr, br, bl], dtype="float32")

print(order_points_new(np.array([1,0,0,1,1,0,0,1]).reshape(4,2)))
order_points_new(np.array([0.2,0.4,0.3,0.2,1,0.2,0.8,1]).reshape(4,2))

[[0. 1.]
 [1. 0.]
 [1. 0.]
 [0. 1.]]


array([[0.3, 0.2],
       [1. , 0.2],
       [0.8, 1. ],
       [0.2, 0.4]], dtype=float32)

In [43]:
# https://gist.github.com/flashlib/e8261539915426866ae910d55a3f9959
def order_points_new(quads):
    # sort the points based on their x-coordinates
    # shape = (box_nums, 4=points_nums, 1), the indices about 4 points
    x_ascend_indices = np.argsort(quads[..., 0:1], axis=1)
    # that's why take_along_axis's argument: axis is 1
    # shape = (box_nums, 4=(x_ascending), 2=(x,y))
    x_ascend = np.take_along_axis(quads, x_ascend_indices, axis=1)

    # grab the left-most and right-most points from the sorted
    # x-roodinate points
    # shape = (box_nums, 2, 2=(x,y))
    leftMost = x_ascend[:, :2]
    rightMost = x_ascend[:, 2:]

    # now, sort the left-most coordinates according to their
    # y-coordinates so we can grab the top-left and bottom-left
    # points, respectively
    # shape = (box_nums, 2=points_nums), the indices about 2 points
    leftMost_y_ascend_indices = np.argsort(leftMost[..., 1:2], axis=1)
    # shape = (box_nums, 2, 2=(x,y))
    leftMost_y_ascend = np.take_along_axis(leftMost, leftMost_y_ascend_indices, axis=1)
    # shape = (box_nums, 1, 2=(x,y))
    tl, bl = leftMost_y_ascend[:, 0:1], leftMost_y_ascend[:, 1:2]
    
    # if use Euclidean distance, it will run in error when the object
    # is trapezoid. So we should use the same simple y-coordinates order method.

    # now, sort the right-most coordinates according to their
    # y-coordinates so we can grab the top-right and bottom-right
    # points, respectively
    # shape = (box_nums, 2=points_nums), the indices about 2 points
    rightMost_y_ascend_indices = np.argsort(rightMost[..., 1:2], axis=1)
    # shape = (box_nums, 2, 2=(x,y))
    rightMost_y_ascend = np.take_along_axis(rightMost, rightMost_y_ascend_indices, axis=1)
    # shape = (box_nums, 1, 2=(x,y))
    tr, br = rightMost_y_ascend[:, 0:1], rightMost_y_ascend[:, 1:2]

    # return the coordinates in top-left, top-right,
    # bottom-right, and bottom-left order
    return np.concatenate([tl, tr, br, bl], axis=1).reshape((-1, 8))

order_points_new(np.array([[1,0,0,1,1,0,0,1],[0.2,0.4,0.3,0.2,1,0.2,0.8,1]]).reshape(-1,4,2))

array([[0. , 1. , 1. , 0. , 1. , 0. , 0. , 1. ],
       [0.3, 0.2, 1. , 0.2, 0.8, 1. , 0.2, 0.4]])